# 03 — EDA: ETF Asset Management

Exploratory analysis of Korean ETFs from an **asset management** standpoint:
return attribution, liquidity, tracking quality, and cross-sectional factor structure.

| Dataset | Source | Coverage |
|---------|--------|----------|
| ETF daily (일별시계열) | `utils.chkxp_ingest` | ~297 ETFs, 2020-10-10 – 2026-02-19, daily |
| Equity OHLCV | `fn_dg6_ingest` | KSE + KOSDAQ, 2016-01-01 – 2026-02-07, daily |
| ETF constituents — KODEX 200 | `fn_dg6_ingest` | 2025-01-01 – 2026-02-07 |
| ETF constituents — TIGER 반도체TOP10 | `fn_dg6_ingest` | 2025-01-01 – 2026-02-18 |

In [1]:
from pathlib import Path

import fn_dg6_ingest
from utils.chkxp_ingest import open_etf_daily

---

## 1. FnGuide DataGuide 6

Daily equity OHLCV and ETF constituent snapshots.

In [2]:
FN_RAW_DIR = Path("../data/raw/fnguide")
FN_DB_DIR  = Path("../data/db/fnguide")

FN_DATASETS = {
    "ohlcv":          FN_RAW_DIR / "dataguide_kse+kosdaq_ohlcv_from(20160101)_to(20260207).csv",
    "etf_const":      FN_RAW_DIR / "dataguide_etfconst(kodex200)_from(20250101)_to(20260207).csv",
    "etf_const_semi": FN_RAW_DIR / "dataguide_etfconst(tiger반도체top10)_from(20250101)_to(20260218).csv",
}

In [3]:
fn_ds = {}
for name, csv_path in FN_DATASETS.items():
    output_dir = FN_DB_DIR / csv_path.stem
    fn_ds[name] = fn_dg6_ingest.open(str(csv_path), output_dir=str(output_dir))
    info = fn_ds[name].describe()
    print(f"{name:16s}  {info.format_name:20s}  {info.shape}  entities={info.entities}")

ohlcv             timeseries_wide       {'default': (7613009, 9)}  entities=4071
etf_const         misc_etf              {'default': (53836, 8)}  entities=0
etf_const_semi    misc_etf              {'default': (2992, 8)}  entities=0


In [4]:
df_ohlcv         = fn_ds["ohlcv"].load()
df_etf_const     = fn_ds["etf_const"].load()
df_etf_const_semi = fn_ds["etf_const_semi"].load()

In [5]:
print("OHLCV:")
print(f"  shape      : {df_ohlcv.shape}")
print(f"  date range : {df_ohlcv['date'].min()} → {df_ohlcv['date'].max()}")
print(f"  securities : {df_ohlcv['코드'].nunique():,}")
df_ohlcv.head()

OHLCV:
  shape      : (7613009, 9)
  date range : 2015-12-30 → 2026-02-06
  securities : 3,071


,코드,코드명,date,수정시가(원),수정고가(원),수정저가(원),수정주가(원),거래량(주),거래대금(원)
0,A000020,동화약품,2015-12-30,8180.0,8180.0,8020.0,8140.0,166761.0,1.348911e+09
1,A000020,동화약품,2016-01-04,8130.0,8150.0,7920.0,8140.0,281440.0,2.265829e+09
2,A000020,동화약품,2016-01-05,8040.0,8250.0,8000.0,8190.0,243179.0,1.981977e+09
3,A000020,동화약품,2016-01-06,8200.0,8590.0,8110.0,8550.0,609906.0,5.129946e+09
4,A000020,동화약품,2016-01-07,8470.0,8690.0,8190.0,8380.0,704752.0,5.919556e+09


In [6]:
print("ETF Constituents — KODEX 200:")
print(f"  shape      : {df_etf_const.shape}")
df_etf_const.head()

ETF Constituents — KODEX 200:
  shape      : (53836, 8)


,date,ETF코드,ETF명,구성종목코드,구성종목,주식수(계약수),금액,금액기준 구성비중(%)
0,2025-01-02,A069500,KODEX 200,,원화현금,NaN,9945243,0.62
1,2025-01-02,A069500,KODEX 200,A000080,하이트진로,47.0,914620,0.06
2,2025-01-02,A069500,KODEX 200,A000100,유한양행,91.0,10765300,0.67
3,2025-01-02,A069500,KODEX 200,A000120,CJ대한통운,17.0,1429700,0.09
4,2025-01-02,A069500,KODEX 200,A000150,두산,11.0,2915000,0.18


In [7]:
print("ETF Constituents — TIGER 반도체TOP10:")
print(f"  shape      : {df_etf_const_semi.shape}")
df_etf_const_semi.head()

ETF Constituents — TIGER 반도체TOP10:
  shape      : (2992, 8)


,date,ETF코드,ETF명,구성종목코드,구성종목,주식수(계약수),금액,금액기준 구성비중(%)
0,2025-01-02,A396500,TIGER 반도체TOP10,,원화현금,NaN,2168653,0.47
1,2025-01-02,A396500,TIGER 반도체TOP10,A058470,리노공업,193.0,38214000,8.25
2,2025-01-02,A396500,TIGER 반도체TOP10,A403870,HPSP,937.0,24783650,5.35
3,2025-01-02,A396500,TIGER 반도체TOP10,A042700,한미반도체,831.0,72380100,15.62
4,2025-01-02,A396500,TIGER 반도체TOP10,A357780,솔브레인,82.0,13366000,2.89


---

## 2. ETF Daily Dataset (일별시계열)

~297 ETFs, daily OHLCV + NAV/IV + tracking metrics, 2020-10 – 2026-02.
Hive-partitioned by `year` at `data/db/chkxp/etf_daily/`.

In [8]:
CHKXP_RAW_DIR = Path("../data/raw/chkxp")

ds_etf = open_etf_daily(
    [
        CHKXP_RAW_DIR / "chkxp_etf(ACE)(KIWOOM)(PLUS)_from(20201010)_to(20260219).csv",
        CHKXP_RAW_DIR / "chkxp_etf(KODEX)_from(20201010)_to(20260219).csv",
        CHKXP_RAW_DIR / "chkxp_etf(RISE)(SOL)_from(20201010)_to(20260219).csv",
        CHKXP_RAW_DIR / "chkxp_etf(TIGER)_from(20201010)_to(20260219).csv",
    ],
    output_dir=Path("../data/db/chkxp/etf_daily"),
)

info = ds_etf.describe()
print(f"format    : {info.format_name}")
print(f"frequency : {info.frequency}")
print(f"shape     : {info.shape[0]:,} rows × {info.shape[1]} cols")
print(f"entities  : {len(info.entities):,} ETFs")
print(f"items     : {info.items}")
print(f"date range: {info.date_range[0]} → {info.date_range[1]}")

format    : daily_etf
frequency : D
shape     : 355,723 rows × 16 cols
entities  : 297 ETFs
items     : ['현재가', '거래대금(30609과동일)', '시가총액', 'ETP지표가치(NAV/IV)', '추적오차율', 'ETP괴리율', 'ETP기초지수현재가', 'ETP구성종목수', '전일현금배당금액', '설정주식수', '전일환매주식수', '전일ETF순자산총액(백만)']
date range: 20200101 → 


In [9]:
df_etf = ds_etf.load()

print(f"shape     : {df_etf.shape}")
print(f"date range: {df_etf['date'].min().date()} → {df_etf['date'].max().date()}")
print(f"entities  : {df_etf['entity_code'].nunique():,} unique ETFs")
print(f"years     : {sorted(df_etf['year'].unique())}")
df_etf.head()

shape     : (355723, 16)
date range: 2020-01-02 → 2026-02-20
entities  : 297 unique ETFs
years     : [np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025), np.int32(2026)]


,date,entity,entity_code,현재가,거래대금(30609과동일),시가총액,ETP지표가치(NAV/IV),추적오차율,ETP괴리율,ETP기초지수현재가,ETP구성종목수,전일현금배당금액,설정주식수,전일환매주식수,전일ETF순자산총액(백만),year
0,2020-01-02,PLUS 한화그룹주,0000J0*001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
1,2020-01-03,PLUS 한화그룹주,0000J0*001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,2020-01-06,PLUS 한화그룹주,0000J0*001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,2020-01-07,PLUS 한화그룹주,0000J0*001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,2020-01-08,PLUS 한화그룹주,0000J0*001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [10]:
df_etf.dtypes

date               datetime64[ms]
entity                        str
entity_code                   str
현재가                       float64
거래대금(30609과동일)            float64
시가총액                      float64
ETP지표가치(NAV/IV)           float64
추적오차율                     float64
ETP괴리율                    float64
ETP기초지수현재가                float64
ETP구성종목수                  float64
전일현금배당금액                  float64
설정주식수                     float64
전일환매주식수                   float64
전일ETF순자산총액(백만)            float64
year                        int32
dtype: object

---

## 3. JKP Factor Returns (Korea, Daily)

Daily **long-short factor return** series for Korea from Jensen, Kelly & Pedersen (2023),
value-weight capped (`vw_cap`) construction.

| File | Description | Shape |
|------|-------------|-------|
| `[kor]_[all_themes]_[daily]_[vw_cap].csv` | All factor themes — columns: `location, name, freq, weighting, n_factors, date, ret` | ~112,636 rows |
| `[kor]_[mkt]_[daily]_[vw_cap].csv` | Market (MKT) return — columns: `location, name, freq, weighting, direction, n_stocks, n_stocks_min, date, ret` | ~9,460 rows |

Both files are in **long format**: one row per `(factor name, date)` observation.

In [12]:
import pandas as pd

JKP_RAW_DIR = Path("../data/raw/jkp")

df_jkp_themes = pd.read_csv(
    JKP_RAW_DIR / "[kor]_[all_themes]_[daily]_[vw_cap].csv",
    parse_dates=["date"],
)

print(f"shape      : {df_jkp_themes.shape}")
print(f"date range : {df_jkp_themes['date'].min().date()} → {df_jkp_themes['date'].max().date()}")
print(f"factors    : {df_jkp_themes['name'].nunique()} unique themes")
print(f"factor list: {sorted(df_jkp_themes['name'].unique())}")
df_jkp_themes.head()

shape      : (112635, 7)
date range : 1988-07-01 → 2024-12-30
factors    : 13 unique themes
factor list: ['accruals', 'debt_issuance', 'investment', 'low_leverage', 'low_risk', 'momentum', 'profit_growth', 'profitability', 'quality', 'seasonality', 'short_term_reversal', 'size', 'value']


,location,name,freq,weighting,n_factors,date,ret
0,kor,accruals,daily,vw_cap,5,1995-05-02,-0.013176
1,kor,accruals,daily,vw_cap,5,1995-05-03,0.014354
2,kor,accruals,daily,vw_cap,5,1995-05-04,0.010243
3,kor,accruals,daily,vw_cap,5,1995-05-08,-0.001226
4,kor,accruals,daily,vw_cap,5,1995-05-09,0.006977


In [13]:
df_jkp_mkt = pd.read_csv(
    JKP_RAW_DIR / "[kor]_[mkt]_[daily]_[vw_cap].csv",
    parse_dates=["date"],
)

print(f"shape      : {df_jkp_mkt.shape}")
print(f"date range : {df_jkp_mkt['date'].min().date()} → {df_jkp_mkt['date'].max().date()}")
df_jkp_mkt.head()

shape      : (9459, 9)
date range : 1986-01-06 → 2023-12-28


,location,name,freq,weighting,direction,n_stocks,n_stocks_min,date,ret
0,kor,mkt,daily,vw_cap,na,7,na,1986-01-29,0.024779
1,kor,mkt,daily,vw_cap,na,3,na,1986-01-25,0.035906
2,kor,mkt,daily,vw_cap,na,4,na,1986-01-28,0.038115
3,kor,mkt,daily,vw_cap,na,5,na,1986-01-31,0.009673
4,kor,mkt,daily,vw_cap,na,6,na,1986-01-30,0.033659
